In [47]:
from tqdm import tqdm_notebook
import pandas as pd
import numpy as np

In [2]:
%%time
import pickle
with open('page_dict.p', 'rb') as fp:
    page_dict = pickle.load(fp)

CPU times: user 5.26 s, sys: 2.42 s, total: 7.68 s
Wall time: 8.59 s


In [3]:
from scipy.spatial.distance import cosine
from sklearn.feature_extraction.text import TfidfVectorizer

In [22]:
vectorizer = TfidfVectorizer(min_df=50, max_df=200, stop_words='english')

In [58]:
200 / 28026

0.007136230642974381

In [23]:
vectors = vectorizer.fit_transform(page_dict.values()).toarray()

In [24]:
vectors.shape

(28026, 37442)

In [25]:
vector_map = dict(zip(page_dict.keys(), vectors.tolist()))

with open('vector_map.p', 'wb') as fp:
    pickle.dump(vector_map, fp)

In [7]:
len(vector_map)

28026

In [26]:
train_data = pd.read_csv('./anomaly-detection-competition-ml1-ts-spring-2020/train_groups.csv')

In [27]:
traingroups_titledata = {}
for i in tqdm_notebook(range(len(train_data))):
    new_doc = train_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title = vector_map[doc_id]
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title, target))

In [44]:
y_train = []
X_train = []
groups_train = []
for new_group in tqdm_notebook(traingroups_titledata):
    docs = traingroups_titledata[new_group]
    for k, (doc_id, title, target_id) in enumerate(docs):
        y_train.append(target_id)
        groups_train.append(new_group)
        all_dist = []
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, target_j = docs[j]
            dist = cosine(title, title_j)
            if dist == None:
                print('aaaaaa')
            all_dist.append(1-dist)
        X_train.append(sorted(all_dist, reverse=True)[0:25]    )

In [56]:
X_train = np.array(X_train)
X_train = np.nan_to_num(X_train)
y_train = np.array(y_train)
groups_train = np.array(groups_train)
print (X_train.shape, y_train.shape, groups_train.shape)

with open('X_train.p', 'wb') as fp:
    pickle.dump(X_train, fp)
    
with open('y_train.p', 'wb') as fp:
    pickle.dump(y_train, fp)

(11690, 25) (11690,) (11690,)


In [50]:
with open('groups_train.p', 'wb') as fp:
    pickle.dump(groups_train, fp)

In [51]:
test_data = pd.read_csv('./anomaly-detection-competition-ml1-ts-spring-2020/test_groups.csv')
testgroups_titledata = {}
for i in tqdm_notebook(range(len(test_data))):
    new_doc = test_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    title = vector_map[doc_id]
    if doc_group not in testgroups_titledata:
        testgroups_titledata[doc_group] = []
    testgroups_titledata[doc_group].append((doc_id, title))

In [55]:
X_test = []
groups_test = []
for new_group in tqdm_notebook(testgroups_titledata):
    docs = testgroups_titledata[new_group]
    for k, (doc_id, title) in enumerate(docs):
        groups_test.append(new_group)
        all_dist = []
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j = docs[j]
            all_dist.append(1-cosine(title, title_j))
        X_test.append(sorted(all_dist, reverse=True)[0:25]    )

In [57]:
X_test = np.array(X_test)
X_test = np.nan_to_num(X_test)
groups_test = np.array(groups_test)
print (X_test.shape, groups_test.shape)

with open('X_test.p', 'wb') as fp:
    pickle.dump(X_test, fp)
    
with open('groups_test.p', 'wb') as fp:
    pickle.dump(groups_test, fp)

(16627, 25) (16627,)
